In [1]:
import numpy as np
import pyvips
import os

from PIL import Image
import matplotlib.pyplot as plt

import pylibczi
from pylibczi import CziScene
import czifile
from czifile import CziFile 

import xml.etree.ElementTree as ET # metadata

## vips_utils.py must be present in the same folder as this czi_script.py
## These scripts are found in the utils folder from the plaquebox-paper (Nature communication)
from utils import vips_utils 


In [2]:
SOURCE_DIR = './'
OUTPUT_DIR = SOURCE_DIR + 'output_czi/'

In [3]:
if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)


image = 'NA5062-04_AB17-24.czi'

In [4]:
img = czifile.imread(SOURCE_DIR + image)
print(img.shape) # (2, 1, 173843, 200355, 3) if the image has two scenes

(2, 1, 166268, 211598, 3)


In [5]:
numpy_array = np.array(img, dtype = np.uint8)
print("Created numpy Array")
#about 5 minutes for this step

scenes = numpy_array.shape[0]
time = numpy_array.shape[1]
height = numpy_array.shape[2]
width = numpy_array.shape[3]
channels = numpy_array.shape[4]

Created numpy Array


In [6]:
numpy_array_reshaped = numpy_array.reshape((height*scenes, width, channels))
print(numpy_array_reshaped.shape) 

(332536, 211598, 3)


In [7]:
dtype_to_format = {
    'uint8': 'uchar',
    'int8': 'char',
    'uint16': 'ushort',
    'int16': 'short',
    'uint32': 'uint',
    'int32': 'int',
    'float32': 'float',
    'float64': 'double',
    'complex64': 'complex',
    'complex128': 'dpcomplex',
}

def numpy2vips(a):
    height, width, bands = a.shape
    linear = a.reshape(width * height * bands)
    vi = pyvips.Image.new_from_memory(linear.data, width, height, bands,
                                      dtype_to_format[str(a.dtype)])
    return vi


In [8]:
vips = numpy2vips(numpy_array_reshaped) 
vips

<pyvips.Image 211598x332536 uchar, 3 bands, rgb>

In [9]:
## Metadata
xml_name = image.split('.')[0]
metadata_name = 'metadata_'+xml_name+'.xml'

czifile = pylibczi.CziFile(SOURCE_DIR + image, metafile_out = OUTPUT_DIR + metadata_name,use_pylibczi=True, verbose=True)
czifile.read_meta()

In [10]:
print('image:', xml_name)
tree = ET.parse(OUTPUT_DIR+metadata_name) 
root = tree.getroot() 
print(root) 

image: NA5062-04_AB17-24
<Element 'ImageDocument' at 0x7f0ee81ed188>


In [11]:
print(ET.tostring(root, encoding='utf8').decode('utf8')) ## display the xml file 

<?xml version='1.0' encoding='utf8'?>
<ImageDocument>
 <Metadata>
  <Experiment Version="1.1">
   <RunMode>ValidateAndAdaptBeforePerformEnabled,OptimizeBeforePerformEnabled,PreventMissingCalibrationDataInformation,PreventStageXYRestoreAfterExperiment,PreventFocusRestoreAfterExperiment</RunMode>
   <BeforeHardwareSetting>HardwareBefore</BeforeHardwareSetting>
   <AfterHardwareSetting>HardwareAfter</AfterHardwareSetting>
   <ExperimentBlockIndex>0</ExperimentBlockIndex>
   <IsSegmented>false</IsSegmented>
   <IsStandardMode>true</IsStandardMode>
   <ImageName>D:\Transfer Folder\20200922 UCD ADC NA 5062\NA5062-4_AB17-24.czi</ImageName>
   <ImageTransferMode>MemoryMappedAndFileStream</ImageTransferMode>
   <AutoSave EnableSingleFileSave="false" IsActivated="false">
    <StorageFolder>D:\zeiss\Pictures</StorageFolder>
    <Name>New</Name>
    <IsAutoSubFolder>true</IsAutoSubFolder>
    <IsAutoSubFolderExternal>true</IsAutoSubFolderExternal>
    <IsClosedOnCompletion>false</IsClosedOnComplet

In [16]:
scale = 0.2 # 0.5 if the image is not too large 
width = int(float(vips.get("width")) * scale) 
height = int(float(vips.get("height")) * scale) 


##### cannot do the codes below becaus PyVips is not 8.6 version
thumbnail = vips.thumbnail_image(width) 
thumbnail


Error: no such operation thumbnail_image
  VipsOperation: class "thumbnail_image" not found


In [15]:
print(image)
vips_utils.show_vips(image)

NA5062-04_AB17-24.czi


-1